# **Neural Net Classifier**

In [ ]:
import pandas as pd
conver = pd.read_csv("/content/triage_dataset_india_expanded.csv")

In [ ]:
!pip install -U langchain-groq langchain langchain-community groq langchain-huggingface transformers torch

In [ ]:
!pip install torch torchvision torchaudio transformers scikit-learn pandas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
# --- Dataset ---
class ESIDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df["Text_Input"].tolist()
        self.ages = df["Age"].astype(float).tolist()
        self.genders = df["Gender"].tolist()
        self.labels = df["Label"].astype(int).tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.gender_encoder = LabelEncoder()
        self.gender_encoder.fit(["Male", "Female", "Other"])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        age = torch.tensor(self.ages[idx], dtype=torch.float)
        gender = torch.tensor(self.gender_encoder.transform([self.genders[idx]])[0], dtype=torch.long)
        label = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # 0-indexed

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors='pt')
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "age": age,
            "gender": gender,
            "label": label
        }

In [ ]:
# --- Model ---
class ESIClassifier(nn.Module):
    def __init__(self, hidden_dim=128, num_classes=5):
        super(ESIClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.gender_embed = nn.Embedding(3, 4)
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + 4 + 1, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, input_ids, attention_mask, age, gender):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        gender_embed = self.gender_embed(gender)
        combined = torch.cat((bert_out, gender_embed, age.unsqueeze(1)), dim=1)
        logits = self.fc(combined)
        return logits

In [ ]:
# --- Training ---
def train_model(df, epochs=5, batch_size=16, lr=2e-5, max_len=128):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    train_df, val_df = train_test_split(df, test_size=0.1, stratify=df["Label"])

    train_ds = ESIDataset(train_df, tokenizer, max_len)
    val_ds = ESIDataset(val_df, tokenizer, max_len)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)

    model = ESIClassifier()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            age = batch["age"].to(device)
            gender = batch["gender"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask, age, gender)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

    torch.save(model.state_dict(), "esi_classifier.pt")
    print("Model saved as esi_classifier.pt")
    return model, tokenizer

In [ ]:
# --- Inference ---
def predict(text, age, gender, model, tokenizer, max_len=128):
    model.eval()
    device = next(model.parameters()).device

    encoding = tokenizer(
        text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=max_len
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    age_tensor = torch.tensor([age], dtype=torch.float).to(device)

    # Gender encoding
    gender_encoder = LabelEncoder()
    gender_encoder.fit(["Male", "Female", "Other"])
    gender_tensor = torch.tensor([gender_encoder.transform([gender])[0]], dtype=torch.long).to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask, age_tensor, gender_tensor)
        probs = torch.softmax(logits, dim=1).squeeze()
        predicted_class = torch.argmax(probs).item()
        confidence = probs[predicted_class].item()

    esi_label = predicted_class + 1  # Adjust if classes are 0-indexed
    return esi_label, confidence

In [ ]:
# --- Entry point ---
if __name__ == "__main__":
    df = pd.read_csv("/content/triage_dataset_india_expanded.csv")
    model, tokenizer = train_model(df)

    # Test
    test_text = "Patient is unconscious and not breathing."
    test_age = 82
    test_gender = "Male"
    label, score = predict(test_text, test_age, test_gender, model, tokenizer)
    print(f"Predicted ESI Level: {label} | Confidence Score: {score:.4f}")


# **LLM Classifier**

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain pdfplumber pypdf torch fitz openai

In [ ]:
# from huggingface_hub import login
# from kaggle_secrets import UserSecretsClient

# # Retrieve secret directly from Kaggle secrets
# user_secrets = UserSecretsClient()
# hf_token = user_secrets.get_secret("HuggingFace_Token")

# # Login to Hugging Face
# login(token=hf_token)


Groq API - Manually Inputing health classification until simra sends her code

In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate


llm = ChatGroq(
    api_key="",
    model_name="llama3-8b-8192",
    temperature=0.2,
    max_tokens=150
)

#triage prompt
triage_prompt = PromptTemplate.from_template("""
You are a highly reliable medical triage assistant. Based on the patient's input and classification (emotional or physical health), assign an Emergency Severity Index (ESI) level from 1 to 5.

The levels are:
- ESI-1: Immediate life-saving intervention required
- ESI-2: High-risk situation, confused/lethargic/disoriented, or severe pain/distress
- ESI-3: Multiple resources needed and vital signs are stable
- ESI-4: One resource needed
- ESI-5: No resources needed

You must provide:
1. ESI Level (1 to 5)
2. A short and specific reason for your decision
3. A confidence score between 0 and 1 indicating how sure you are about the ESI level

Respond in the following format:
ESI Level: [1-5]
Reason: [Short explanation]
Confidence: [float between 0 and 1]

Classification: {classification}
Symptoms: {patient_input}
""")

# Triage function
def llm_triage(classification, patient_input):
    triage_chain = triage_prompt | llm
    triage_output = triage_chain.invoke({
    "classification": classification,
    "patient_input": patient_input
    }).content.strip()

    print(f"\n[DEBUG] LLM Output:\n{triage_output}")
    return triage_output

# Example call
output = llm_triage(
    classification="physical health",
    patient_input="Patient is experiencing little nausea due to alcohol consumption."
)

# **MentalHealth or PhysicalHealth Classifier**

In [ ]:
# === Imports ===
import re
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# === Load Emotion Lexicon ===
emotion_df = pd.read_csv("/content/emotion_lexicon.csv")  # NRC Lexicon
emotion_df['word'] = emotion_df['word'].str.lower().str.strip()
emotion_df.set_index('word', inplace=True)
emotion_dict = emotion_df.to_dict(orient='index')

def compute_emotion_scores(text, emotion_dict):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    emotion_categories = list(next(iter(emotion_dict.values())).keys())
    scores = dict.fromkeys(emotion_categories, 0)
    matched_words = {}

    for token in tokens:
        if token in emotion_dict:
            matched_words[token] = emotion_dict[token]
            for emotion in emotion_categories:
                scores[emotion] += emotion_dict[token][emotion]
    return scores, matched_words

# === Load MentalBERT Model ===
model_checkpoint = "mental/mental-bert-base-uncased"
weights_path = "/content/mentalbert_weights.pth"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_auth_token=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, use_auth_token=True)
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

label_map = {0: "Physical Health", 1: "Mental Health"}

# === Final Decision Function ===
def classify_with_emotion_fusion(text):
    # --- MentalBERT prediction ---
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        mental_conf = probs[0][1].item()
        physical_conf = probs[0][0].item()

    # --- Emotion Scores ---
    scores, matched = compute_emotion_scores(text, emotion_dict)
    negative_emotions = ['negative', 'fear', 'sadness', 'anger', 'disgust']
    dom_emotions = [emo for emo, val in scores.items() if val == max(scores.values()) and val > 0]
    is_negative_dominant = any(emo in dom_emotions for emo in negative_emotions)

    # --- Logic Fusion ---
    if mental_conf > 0.7 and is_negative_dominant:
        decision = "Mental Health Issue"
    elif physical_conf > 0.7 and is_negative_dominant:
        decision = "Physical Health Issue"
    elif mental_conf < 0.6 and physical_conf < 0.6:
        decision = "Uncertain – Need more input or clarification"
    else:
        decision = f"Model Suggests: {label_map[pred_class]} (Confidence: {max(mental_conf, physical_conf):.2f})"

    return {
        "text": text,
        "mental_conf": mental_conf,
        "physical_conf": physical_conf,
        "emotion_scores": scores,
        "dominant_emotions": dom_emotions,
        "final_decision": decision
    }


In [ ]:
text_input = "I am in my pms period and feeling angry"

result = classify_with_emotion_fusion(text_input)

# === Print Results ===
print("\n Final Analysis")
print("Text:", result['text'])
print("MentalBERT Confidence → Mental:", round(result['mental_conf'], 3), ", Physical:", round(result['physical_conf'], 3))
print("Dominant Emotions:", result['dominant_emotions'])
print("Final Decision:", result['final_decision'])


# ***Inference Code***

In [ ]:
# === Imports ===
import re
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

emotion_df = pd.read_csv("/content/emotion_lexicon.csv")
emotion_df['word'] = emotion_df['word'].str.lower().str.strip()
emotion_df.set_index('word', inplace=True)
emotion_dict = emotion_df.to_dict(orient='index')


def compute_emotion_scores(text, emotion_dict):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    emotion_categories = list(next(iter(emotion_dict.values())).keys())
    scores = dict.fromkeys(emotion_categories, 0)
    matched_words = {}

    for token in tokens:
        if token in emotion_dict:
            matched_words[token] = emotion_dict[token]
            for emotion in emotion_categories:
                scores[emotion] += emotion_dict[token][emotion]
    return scores, matched_words

# === Load MentalBERT Model ===
model_checkpoint = "mental/mental-bert-base-uncased"
weights_path = "/content/mentalbert_weights.pth"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_auth_token=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, use_auth_token=True)
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

label_map = {0: "Physical Health", 1: "Mental Health"}

# === Function: Classify with Emotion Fusion ===
def classify_with_emotion_fusion(text):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        mental_conf = probs[0][1].item()
        physical_conf = probs[0][0].item()

    # --- Emotion Scores ---
    scores, matched = compute_emotion_scores(text, emotion_dict)
    negative_emotions = ['negative', 'fear', 'sadness', 'anger', 'disgust']
    dom_emotions = [emo for emo, val in scores.items() if val == max(scores.values()) and val > 0]
    is_negative_dominant = any(emo in dom_emotions for emo in negative_emotions)

    # --- Fusion Logic ---
    if mental_conf > 0.7 and is_negative_dominant:
        decision = "Mental Health Issue"
    elif physical_conf > 0.7 and is_negative_dominant:
        decision = "Physical Health Issue"
    elif mental_conf < 0.6 and physical_conf < 0.6:
        decision = "Uncertain – Need more input or clarification"
    else:
        decision = f"Model Suggests: {label_map[pred_class]} (Confidence: {max(mental_conf, physical_conf):.2f})"

    return {
        "text": text,
        "mental_conf": mental_conf,
        "physical_conf": physical_conf,
        "emotion_scores": scores,
        "dominant_emotions": dom_emotions,
        "final_decision": decision
    }

llm = ChatGroq(
    api_key="",
    model_name="llama3-8b-8192",
    temperature=0.2,
    max_tokens=150
)

# === Triage Prompt Template ===
triage_prompt = PromptTemplate.from_template("""
You are a highly reliable medical triage assistant. Based on the patient's input and classification input(emotional or physical health), assign an Emergency Severity Index (ESI) level from 1 to 5 based on the classification.

The levels are:
- ESI-1: Immediate life-saving intervention required
- ESI-2: High-risk situation, confused/lethargic/disoriented, or severe pain/distress
- ESI-3: Multiple resources needed and vital signs are stable
- ESI-4: One resource needed
- ESI-5: No resources needed

You must provide:
1. ESI Level (1 to 5)
2. A short and specific reason for your decision
3. A confidence score between 0 and 1 indicating how sure you are about the ESI level

Respond in the following format:
ESI Level: [1-5]
Reason: [Short explanation]
Confidence: [float between 0 and 1]

Classification: {classification}
Symptoms: {patient_input}
""")

# === Function: LLM-based Triage ===
def llm_triage(classification, patient_input):
    triage_chain = triage_prompt | llm
    triage_output = triage_chain.invoke({
        "classification": classification,
        "patient_input": patient_input
    }).content.strip()

    print(f"\n[DEBUG] LLM Output:\n{triage_output}")
    return triage_output

# === Full Pipeline Function ===
def run_full_pipeline(patient_input):

    classification_output = classify_with_emotion_fusion(patient_input)
    if "Mental Health" in classification_output['final_decision']:
        classification_for_triage = "mental health"
    elif "Physical Health" in classification_output['final_decision']:
        classification_for_triage = "physical health"
    else:
        classification_for_triage = "uncertain"


    triage_output = llm_triage(
        classification=classification_for_triage,
        patient_input=patient_input
    )


    return {
        "classification_output": classification_output,
        "triage_output": triage_output
    }

# === Example Execution ===
if __name__ == "__main__":
    patient_input = "Patient is experiencing little nausea due to alcohol consumption."
    full_output = run_full_pipeline(patient_input)

    # print("\n=== Final Outputs ===")
    print(f"Classification Output:\n{full_output['classification_output']}")
    print(f"\nTriage Output:\n{full_output['triage_output']}")


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import LabelEncoder

# --- Model ---
class ESIClassifier(nn.Module):
    def __init__(self, hidden_dim=128, num_classes=5):
        super(ESIClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.gender_embed = nn.Embedding(3, 4)
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + 4 + 1, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, input_ids, attention_mask, age, gender):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        gender_embed = self.gender_embed(gender)
        combined = torch.cat((bert_out, gender_embed, age.unsqueeze(1)), dim=1)
        logits = self.fc(combined)
        return logits

# --- Inference ---
def predict_esi(
    model_path,
    test_text="Patient is unconscious and not breathing.",
    test_age=82,
    test_gender="Male",
    max_len=128
):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ESIClassifier().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    encoding = tokenizer(
        test_text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=max_len
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    age_tensor = torch.tensor([test_age], dtype=torch.float).to(device)

    gender_encoder = LabelEncoder()
    gender_encoder.fit(["Male", "Female", "Other"])
    gender_tensor = torch.tensor([gender_encoder.transform([test_gender])[0]], dtype=torch.long).to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask, age_tensor, gender_tensor)
        probs = torch.softmax(logits, dim=1).squeeze()
        predicted_class = torch.argmax(probs).item()
        confidence = probs[predicted_class].item()

    esi_label = predicted_class + 1
    print(f"\nPredicted ESI Level: {esi_label} | Confidence Score: {confidence:.4f}")
    return esi_label, confidence


In [ ]:
predict_esi("esi_classifier.pt", test_text = "The patient is unresponsive, not breathing, and has no detectable pulse.", test_age=89, test_gender="Female")


In [ ]:
def resolve_esi_via_confidence(patient_text, age, gender, model_path):
    # --- Neural Network Inference ---
    esi_nn, conf_nn = predict_esi(
        model_path=model_path,
        test_text=patient_text,
        test_age=age,
        test_gender=gender
    )
    print(f"[Neural Network] Predicted ESI Level: {esi_nn} | Confidence Score: {conf_nn:.4f}")

    # --- LLM Inference ---
    llm_result = run_full_pipeline(patient_text)
    llm_output = llm_result['triage_output']
    print("\n[LLM Output]")
    print(llm_output.strip())

    # --- Parse LLM Results ---
    try:
        lines = llm_output.splitlines()
        esi_llm = int([line for line in lines if line.startswith("ESI Level")][0].split(":")[1].strip())
        conf_llm = float([line for line in lines if line.startswith("Confidence")][0].split(":")[1].strip())
    except Exception as e:
        print(f"[ERROR] Failed to parse LLM output: {e}")
        esi_llm, conf_llm = -1, 0.0

    # --- Final Decision ---
    if conf_nn >= conf_llm:
        final_method = "Neural Network"
        final_esi = esi_nn
        final_conf = conf_nn
    else:
        final_method = "LLM"
        final_esi = esi_llm
        final_conf = conf_llm

    print(f"\n[Final Decision] Method Used: {final_method} | ESI Level: {final_esi} | Confidence: {final_conf:.4f}")

    return {
        "final_esi": final_esi,
        "final_confidence": final_conf,
        "selected_method": final_method,
        "llm_output": llm_output,
        "classification_output": llm_result["classification_output"]
    }

In [ ]:
result = resolve_esi_via_confidence(
    patient_text="The patient is not breathing, and has less detectable pulse.",
    age=89,
    gender="Female",
    model_path="esi_classifier.pt"
)

print(f"\n[FINAL DECISION] ESI: {result['final_esi']} (Confidence: {result['final_confidence']:.2f}) via {result['selected_method']}")

# ***Lab Reports Input***

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F
import re
import pandas as pd
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate


emotion_df = pd.read_csv("emotion_lexicon.csv")
emotion_df['word'] = emotion_df['word'].str.lower().str.strip()
emotion_df.set_index('word', inplace=True)
emotion_dict = emotion_df.to_dict(orient='index')

def compute_emotion_scores(text, emotion_dict):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    categories = list(next(iter(emotion_dict.values())).keys())
    scores = dict.fromkeys(categories, 0)
    for t in tokens:
        if t in emotion_dict:
            for emo, val in emotion_dict[t].items():
                scores[emo] += val
    return scores

# Load MentalBERT
MBERT_CHECKPOINT = "mental/mental-bert-base-uncased"
tokenizer_mbert = AutoTokenizer.from_pretrained(MBERT_CHECKPOINT, use_auth_token=True)
model_mbert = AutoModelForSequenceClassification.from_pretrained(MBERT_CHECKPOINT, num_labels=2, use_auth_token=True)
model_mbert.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_mbert.to(device)
label_map = {0: "Physical Health", 1: "Mental Health"}

def classify_with_emotion_fusion(text: str):
    enc = tokenizer_mbert(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        out = model_mbert(**enc)
        probs = F.softmax(out.logits, dim=1).squeeze()
        phys_conf, ment_conf = float(probs[0]), float(probs[1])
        cls_idx = int(torch.argmax(probs).item())
    emo_scores = compute_emotion_scores(text, emotion_dict)
    neg_emotions = {'negative','fear','sadness','anger','disgust'}
    dom = [e for e,v in emo_scores.items() if v==max(emo_scores.values()) and v>0]
    neg_dom = any(e in neg_emotions for e in dom)
    if ment_conf > .7 and neg_dom:
        final = "Mental Health"
    elif phys_conf > .7 and neg_dom:
        final = "Physical Health"
    elif ment_conf < .6 and phys_conf < .6:
        final = "Uncertain"
    else:
        final = label_map[cls_idx]
    return {"physical_conf": phys_conf, "mental_conf": ment_conf, "final_decision": final}

# === Setup Groq LLM ===
llm = ChatGroq(api_key="", model_name="llama3-8b-8192", temperature=0.2, max_tokens=150)
triage_prompt = PromptTemplate.from_template("""
You are a highly reliable medical triage assistant.
Based on the patient's input, classification (emotional or physical health), and lab results, assign an Emergency Severity Index (ESI) level from 1 to 5.

The levels are:
- ESI-1: Immediate life-saving intervention required
- ESI-2: High-risk situation, confused/lethargic/disoriented, or severe pain/distress
- ESI-3: Multiple resources needed and vital signs are stable
- ESI-4: One resource needed
- ESI-5: No resources needed

Provide:
1. ESI Level: [1–5]
2. Reason: [Short explanation]
3. Confidence: [float between 0 and 1]

Classification: {classification}
Symptoms: {symptoms}
Lab Results:
{lab_results}
""")

def llm_triage(classification, symptoms, lab_results):
    chain = triage_prompt | llm
    out = chain.invoke({"classification": classification, "symptoms": symptoms, "lab_results": lab_results})
    return out.content.strip()

symptoms = "Patient is experiencing little nausea due to alcohol consumption."
labs = {
    "Fasting Glucose": 205.0,
    "Postprandial Glucose": 299.0,
    "Total Cholesterol": 272.0,
    "Triglycerides": 174.0,
    "Insulin Level": 17.3
}
lab_str = "\n".join(f"{k}: {v}" for k, v in labs.items())
fusion_out = classify_with_emotion_fusion(f"{symptoms}\nLab Results:\n{lab_str}")
print("Emotion-Fusion Output:", fusion_out)

# Run the LLM triage
tag = fusion_out["final_decision"].lower().replace(" ", " ")
llm_out = llm_triage(tag, symptoms, lab_str)
print("\nLLM Triage Output:\n", llm_out)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F
import re
import pandas as pd
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

# 1) Emotion-Fusion Setup (unchanged) …
#    [load emotion_lexicon.csv, MentalBERT, compute_emotion_scores(), classify_with_emotion_fusion()]


llm = ChatGroq(
    api_key="",
    model_name="llama3-8b-8192",
    temperature=0.2,
    max_tokens=200
)

triage_prompt = PromptTemplate.from_template("""
You are a highly reliable medical triage assistant.
Based on the patient's input, classification (emotional or physical health), lab results, diagnosis, and treatment plan, assign an Emergency Severity Index (ESI) level from 1 to 5.

The levels are:
- ESI-1: Immediate life-saving intervention required
- ESI-2: High-risk situation, confused/lethargic/disoriented, or severe pain/distress
- ESI-3: Multiple resources needed and vital signs are stable
- ESI-4: One resource needed
- ESI-5: No resources needed

Provide:
1. ESI Level: [1–5]
2. Reason: [Short explanation]
3. Confidence: [float between 0 and 1]

Classification: {classification}
Symptoms: {symptoms}
Lab Results:
{lab_results}
Diagnosis: {diagnosis}
Treatment Plan: {treatment_plan}
""")

def llm_triage(classification: str,
               symptoms: str,
               lab_results: str,
               diagnosis: str,
               treatment_plan: str) -> str:
    chain = triage_prompt | llm
    out = chain.invoke({
        "classification": classification,
        "symptoms": symptoms,
        "lab_results": lab_results,
        "diagnosis": diagnosis,
        "treatment_plan": treatment_plan
    })
    return out.content.strip()

# 3) Manual Test
if __name__ == "__main__":

    symptoms = "Patient is experiencing nausea and mild abdominal pain."
    labs = {
        "Fasting Glucose": 105.0,
        "Postprandial Glucose": 140.0,
        "Total Cholesterol": 180.0,
        "Triglycerides": 150.0,
        "Insulin Level": 12.5
    }
    lab_str = "\n".join(f"{k}: {v}" for k, v in labs.items())
    fused = classify_with_emotion_fusion(f"{symptoms}\nLab Results:\n{lab_str}")
    print("Emotion-Fusion Output:", fused)
    tag = fused["final_decision"].lower().replace(" ", " ")
    diagnosis      = "Acute gastritis"
    treatment_plan = "Begin Proton Pump Inhibitor therapy, advise bland diet, rehydrate."

    # d) Run LLM triage
    llm_out = llm_triage(tag, symptoms, lab_str, diagnosis, treatment_plan)
    print("\nLLM Triage Output:\n", llm_out)

In [ ]:
# import re
# import requests
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# from langchain_groq import ChatGroq
# from langchain.prompts import PromptTemplate

# # === 1) Emotion-Fusion Setup ===
# # Load emotion lexicon
# emotion_df = pd.read_csv("emotion_lexicon.csv")
# emotion_df['word'] = emotion_df['word'].str.lower().str.strip()
# emotion_df.set_index('word', inplace=True)
# emotion_dict = emotion_df.to_dict(orient='index')

# def compute_emotion_scores(text, emotion_dict):
#     text = text.lower()
#     tokens = re.findall(r'\b\w+\b', text)
#     categories = next(iter(emotion_dict.values())).keys()
#     scores = dict.fromkeys(categories, 0)
#     for t in tokens:
#         if t in emotion_dict:
#             for emo, val in emotion_dict[t].items():
#                 scores[emo] += val
#     return scores

# # Load MentalBERT
# MBERT_CHECKPOINT = "mental/mental-bert-base-uncased"
# MBERT_WEIGHTS    = "mentalbert_weights.pth"

# tokenizer_mbert = AutoTokenizer.from_pretrained(MBERT_CHECKPOINT, use_auth_token=True)
# model_mbert     = AutoModelForSequenceClassification.from_pretrained(
#     MBERT_CHECKPOINT, num_labels=2, use_auth_token=True
# )
# model_mbert.load_state_dict(torch.load(MBERT_WEIGHTS, map_location="cpu"))
# model_mbert.eval()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_mbert.to(device)
# label_map = {0: "Physical Health", 1: "Mental Health"}

# def classify_with_emotion_fusion(text: str) -> dict:
#     enc = tokenizer_mbert(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     enc = {k: v.to(device) for k, v in enc.items()}
#     with torch.no_grad():
#         out   = model_mbert(**enc)
#         probs = F.softmax(out.logits, dim=1).squeeze()
#         phys, ment = float(probs[0]), float(probs[1])
#         cls_idx    = int(torch.argmax(probs).item())

#     emo_scores = compute_emotion_scores(text, emotion_dict)
#     neg_em    = {'negative','fear','sadness','anger','disgust'}
#     dom       = [e for e,v in emo_scores.items() if v==max(emo_scores.values()) and v>0]
#     neg_dom   = any(e in neg_em for e in dom)

#     if ment > .7 and neg_dom:
#         final = "Mental Health"
#     elif phys > .7 and neg_dom:
#         final = "Physical Health"
#     elif ment < .6 and phys < .6:
#         final = "Uncertain"
#     else:
#         final = label_map[cls_idx]

#     return {
#         "physical_conf": phys,
#         "mental_conf": ment,
#         "emotion_scores": emo_scores,
#         "dominant_emotions": dom,
#         "final_decision": final
#     }

# # === 2) Groq LLM Triage Setup ===
# llm = ChatGroq(
#     api_key="YOUR_GROQ_KEY",
#     model_name="llama3-8b-8192",
#     temperature=0.2,
#     max_tokens=200
# )

# triage_prompt = PromptTemplate.from_template("""
# You are a highly reliable medical triage assistant.
# Based on the patient's input, classification (emotional or physical health), lab results, diagnosis, and treatment plan, assign an Emergency Severity Index (ESI) level from 1 to 5.

# The levels are:
# - ESI-1: Immediate life-saving intervention required
# - ESI-2: High-risk situation, confused/lethargic/disoriented, or severe pain/distress
# - ESI-3: Multiple resources needed and vital signs are stable
# - ESI-4: One resource needed
# - ESI-5: No resources needed

# Provide:
# 1. ESI Level: [1–5]
# 2. Reason: [Short explanation]
# 3. Confidence: [float between 0 and 1]

# Classification: {classification}
# Symptoms: {symptoms}
# Lab Results:
# {lab_results}
# Diagnosis: {diagnosis}
# Treatment Plan: {treatment_plan}
# """)

# def llm_triage(classification, symptoms, lab_results, diagnosis, treatment_plan) -> str:
#     chain = triage_prompt | llm
#     resp  = chain.invoke({
#         "classification": classification,
#         "symptoms":      symptoms,
#         "lab_results":   lab_results,
#         "diagnosis":     diagnosis,
#         "treatment_plan":treatment_plan
#     })
#     return resp.content.strip()

# # === 3) Data Fetch & Pipeline ===
# def fetch_patient_data(patient_id: str) -> dict:
#     url  = f"https://your-api/patient/{patient_id}"
#     resp = requests.get(url)
#     resp.raise_for_status()
#     return resp.json()

# def run_full_pipeline(patient_id: str) -> dict:
#     data     = fetch_patient_data(patient_id)
#     symptoms = data["symptoms"]
#     labs     = data.get("labs", {})
#     lab_str  = "\n".join(f"{k}: {v}" for k,v in labs.items())
#     diagnosis      = data.get("diagnosis", "")
#     treatment_plan = data.get("treatment_plan", "")

#     # 1) Emotion-fusion tag
#     fused = classify_with_emotion_fusion(f"{symptoms}\nLab Results:\n{lab_str}")
#     tag   = fused["final_decision"].lower().replace(" ", " ")

#     # 2) LLM triage
#     triage_out = llm_triage(tag, symptoms, lab_str, diagnosis, treatment_plan)

#     # 3) Parse LLM
#     lines   = triage_out.splitlines()
#     try:
#         esi_llm  = int([l for l in lines if l.startswith("ESI Level")][0].split(":")[1])
#         conf_llm = float([l for l in lines if l.startswith("Confidence")][0].split(":")[1])
#     except:
#         esi_llm, conf_llm = -1, 0.0

#     # 4) Compare confidences
#     conf_nn = max(fused["physical_conf"], fused["mental_conf"])
#     esi_nn  = 1 if fused["final_decision"]=="Physical Health" else 2 if fused["final_decision"]=="Mental Health" else 3

#     final = (
#       {"method":"Neural+Fusion","esi":esi_nn,"confidence":conf_nn}
#       if conf_nn>=conf_llm
#       else {"method":"LLM","esi":esi_llm,"confidence":conf_llm}
#     )

#     return {
#       "patient_id":      patient_id,
#       "fusion_output":   fused,
#       "llm_output":      triage_out,
#       "final_decision":  final
#     }

# # === 4) Manual Test ===
# if __name__=="__main__":
#     pid = "123"
#     result = run_full_pipeline(pid)
#     import json
#     print(json.dumps(result, indent=2))
